<font color='red'>
    <ul>
        <li>update tasks / descriptions of what to do</li>
        <li>add illustrations</li>
        <li>remove TQDM</li>
    </ul>
</font>

This is a tutorial notebook on object-oriented software design, prepared for the Foundations of Software Engineering (FSE v2020.1, https://github.com/adasegroup/FSE2020_seminars) at Skoltech (http://skoltech.ru).

Copyright 2020 by Alexey Artemov and ADASE Lab. 

# FSE-08: Object-Oriented Software Design

The purpose of this practical is to give an introduction into object-oriented software design. You will learn to:
 - Formulate real-world problems in object-oriented terms
 - Use Python classes to implement the designed models

### Legend

In these frightening times, it is of utmost importance to be able to model the spread of various viruses. Our government has hired your company to create a flexible computational model for the spread of the most dangerous viruses, the Virus Spread Modelling System VSMS-20.

In this task, your goal is to design an implement models of these viruses:

 - Seasonal flu virus
 - SARS-CoV-2
 - Cholera 
 
The goal is to be able to perform simulations to assess efficiency of public policymaking aiming to save lives.

**What do we need to model?**

In this design, we need to model the following domain entities as objects:
 1. Viruses and infections, responsible for infecting persons.
 2. Persons, responsible for being infected, transmitting infections.
 3. The Department of Health, responsible for healthcare policymaking.
 4. Hospitals, responsible for treating persons and performing tests.
 
We need to model the following processes:
 1. Person getting infected by the virus.
 2. Person going through a number of states (e.g. healthy - asymptomatic infected - symptomatic infected - recovered).
 3. Person getting tested for a speficic type of virus.
 4. Person getting hospitalized.
 5. Hospital figuring out the treatment for a person based on tests.
 6. Department of Health checking for the number of infections, as reported by positive tests.
 7. Department of Health establishing a policy (e.g., social distancing, full/partial lockdown, or mandatory wearing masks).

We assume the following environmental constraints:
 - Persons cannot be born during the course of the simulation (but may become dead)
 - Persons can only have 1 virus (but many antibodies)
 - Persons may or may not obey the policies of the government
 - The Department of Health does not penalize persons for not obeying the policies

**The purpose of this part of the practical** is to figure out the preliminary design of such a system. 

---

### 1. Model viruses/infections

Define the abstract class `Infectable` with a method `infect`. 
 1. Model the virus "strength", i.e. the ability of the virus to cause more severe symptoms.
 2. Model the virus "contagiousness", i.e. the ability of the virus to transmit from person to person.

In [44]:
from abc import ABC, abstractmethod

In [141]:
class Infectable(ABC):
    def __init__(self, strength=1.0, contag=1.0):
        # contag is for contagiousness so we have less typos
        self._strength = strength
        self._contag = contag
        
    @abstractmethod
    def infect(self, person):
        pass

In [142]:
class SeasonalFluVirus(Infectable):
    def infect(self, person):
        pass

In [143]:
class SARSCoV2(Infectable):
    def infect(self, person):
        pass

In [144]:
class Cholera(Infectable):
    def infect(self, person):
        pass

### 2. Model a person

**Task 2.1.:** 
 1. Use abstraction to identify the possible fields that are relevant to the epidemiologic context.
 2. Create interfaces for methods describing person's basic daily activities (e.g., morning trip from home to work, afternoon work, evening trip from work to home, and nightly sleep)
 3. Create interfaces for methods describing person's basic interactions. 
 4. Create implementations for functions where they are obvious.

In [151]:
class Person:
    def __init__(self, home_position=(0, 0), age=30, weight=70):
        self._virus = None
        self._antibodies = []
        self._temperature = 36.6
        self.weight = weight
        self._water = 0.6 * self.weight
        self.age = age
        self._home_position = home_position
        self.position = home_position
    
    def day_actions(self):
        self.position = (randint(min_j, max_j), randint(min_i, max_i))

    def night_actions(self):
        self.position = self._home_position

    def interact(self, other):
        pass

    def get_infected(self, virus):
        # check if we are not infected
        self._virus = virus
    
    def is_close_to(self, other):
        return self.position == other.position
    
    @property
    def temperature(self):
        return self._temperature
    
    @property
    def water(self):
        return self._water

### 3. Model of a hospital

**Task 3.1.:** 
 1. Use abstraction to identify the possible Hospital fields that are relevant to the epidemiologic context.
 2. Create interfaces for methods responsible for Hospital basic daily activities (e.g., identifying type of disease and treating patients).
 3. Create implementations for functions where they are obvious.

In [93]:
class Hospital:
    def __init__(self, capacity):
        self._capacity = capacity
        self._persons = []
    
    def treat_person(self, person):
        pass
    
    def identify_disease(self, person: Person) -> AbstractVirus:
        pass

### 4. Model of the Department of Health

**Task 4.1.:** 
 1. Use abstraction to identify the possible Government fields that are relevant to the epidemiologic context.
 2. Create interfaces for methods responsible for Government basic daily activities (e.g., identifying type of situation and issuing policies).
 3. Create implementations for functions where they are obvious.

In [99]:
class DepartmentOfHealth:
    def __init__(self):
        pass
    
    def monitor_situation(self):
        pass
    
    def issue_policy(self):
        pass

### 5. Model the world

The basic algorithm works on a daily basis.
 1. The State counts positive/negative tests from yesterday, decides if they need to introduce the new policies/restrictions.
 2. The hospitals treat patients.
 3. The new tests are made by random persons and for all persons in hospitals.
 4. People do their daily activities and interact, possibly infecting each other.
 5. 

In [133]:
def simulate_day(persons, health_dept, hospitals):
    health_dept.monitor_situation()
    health_dept.issue_policy()
    
    for hospital in hospitals:
        hospital.treat_patients()
    
    for person in persons:
        person.day_actions()
    
    for person in persons:
        for other in persons:
            if person is not other and person.is_close_to(other):
                person.interact(other)
                
    for person in persons:
        person.night_actions()

We initialise our 

In [139]:
from random import randint


def create_persons(min_j, max_j, min_i, max_i, n_persons):
    min_age, max_age = 1, 90
    min_weight, max_weight = 30, 120
    persons = [
        Person(
            home_position=(randint(min_j, max_j), randint(min_i, max_i)),
            age=randint(min_age, max_age),
            weight=randint(min_weight, max_weight),
        )
        for i in range(n_persons)
    ]
    return persons


def create_department_of_health():
    return DepartmentOfHealth()


def create_hospitals(n_hospitals):
    hospitals = [
        Hospital(capacity=100)
        for i in range(n_hospitals)
    ]
    return hospitals


def initialize():
    # our little country
    min_i, max_i = 0, 100
    min_j, max_j = 0, 100
    
    # our citizen    
    n_persons = 1000
    persons = create_persons(min_j, max_j, min_i, max_i, n_persons)
    
    # our health department
    health_dept = create_department_of_health()
    
    # our healthcare system
    n_hospitals = 4
    hospitals = create_hospitals(n_hospitals)

    return persons, health_dept, hospitals

In [137]:
import tqdm

In [138]:
persons, health_dept, hospitals = initialize()

for day in tqdm.tqdm(range(100)):
    simulate_day(persons, health_dept, hospitals)

100%|██████████| 100/100 [00:28<00:00,  3.47it/s]
